In [8]:
# all imports
import numpy as np
import os
import requests
from astropy.table import Table, vstack
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import matplotlib.patches as mpatches
import matplotlib.lines as mlines
from astropy.io import fits
from astropy.coordinates import SkyCoord
from astropy import units as u
from astropy.wcs import WCS
import math
import matplotlib as mpl
from astropy.visualization.wcsaxes import SphericalCircle
import pandas as pd
import random
from matplotlib.patches import Ellipse
import ligo.skymap.plot
from scipy.optimize import minimize
import numdifftools as ndt
from scipy.interpolate import griddata
from matplotlib.tri import Triangulation
from matplotlib.colors import LogNorm
from matplotlib.pyplot import *

In [16]:
# define directories
data_dir = '/global/cfs/projectdirs/desi/science/td/pv/tfgalaxies/'
cache_dir = '/pscratch/sd/j/jjpim/cache/'
scratch_dir = '/pscratch/sd/j/jjpim/'
v_dir = '/pscratch/sd/j/jjpim/massmag/'
hist_dir = '/pscratch/sd/j/jjpim/'

# load updated master table
tf_mastertable = Table.read(data_dir + 'tf_mastertable_massmag_stellar.fits') 

#load SGA table
SGA = Table.read('/global/cfs/cdirs/cosmo/data/sga/2020/SGA-2020.fits', 'ELLIPSE')

#load H1 index
h1_table = Table.read('/global/homes/k/kadglass/DESI_SGA/data/SGA-2020_ALFALFA.fits')
h1[:5]

#load H1 data
h1_mass = Table.read('/pscratch/sd/j/jjpim/a100.code12.table2.190808.csv')

In [17]:
h1_mass[:5]

AGCNr,Name,RAdeg_HI,DECdeg_HI,RAdeg_OC,DECdeg_OC,Vhelio,W50,sigW,W20,HIflux,sigflux,SNR,RMS,Dist,sigDist,logMH,siglogMH,HIcode
int64,str8,float64,float64,float64,float64,int64,int64,int64,int64,float64,float64,float64,float64,float64,float64,float64,float64,int64
105367,........,0.00167,5.44333,0.00333,5.4425,11983,274,39,281,1.14,0.08,8.1,1.91,166.0,2.3,9.87,0.05,1
333313,........,0.00375,24.90889,359.9975,24.9075,11181,313,20,333,1.8,0.09,11.3,2.02,154.8,2.3,10.01,0.05,1
331060,478-009b,0.01042,23.08472,0.01417,23.0875,4463,160,4,184,1.96,0.07,14.7,2.35,50.6,10.4,9.07,0.18,1
331061,456-013.,0.01042,15.87222,0.00875,15.88167,6007,260,45,268,1.13,0.09,6.5,2.4,85.2,2.4,9.29,0.06,1
104570,........,0.00667,32.70834,0.005,32.71028,10614,245,6,250,0.86,0.07,6.6,1.86,147.0,2.3,9.64,0.06,1


In [ ]:
#new column in tf mastertable for H1 masses
tf_mastertable['M_HI']=np.nan
tf_mastertable['M_HI_ERR']=np.nan

In [ ]:
# put target ids in list if mass exists
sgas = []
sga_ids = tf_mastertable['SGA_ID'][tf_mastertable['M_R26'] > 0]

# find matching target ids for the first table
for sga_id in sga_ids:
    if len(h1_table[h1_table['SGA_ID'] == sga_id]) > 0:
        targets.append(sga_id)

#extract H1 data and update to mastertable
for sga_id in sga_ids:
    h1_mass = h1_table[h1_table['SGA_ID'] == sga_id]['LOGM'][0] #ask what columns are h1 mass!!!
    h1_mass_err = h1_table[h1_table['SGA_ID'] == sga_id]['LOGM_ERR'][0] #ask what column is h1 error
    tf_mastertable['M_H1'][np.logical_and(tf_mastertable['SGA_ID'] == sga_id, tf_mastertable['M_R26'] > 0)] = h1_mass
    tf_mastertable['M_H1_ERR'][np.logical_and(tf_mastertable['SGA_ID'] == sga_id, tf_mastertable['M_R26'] > 0)] = h1_mass_err